In [1]:
################################################################
# import blocks
################################################################
# I/O
import os
import numpy as np
import os.path
import operator
import copy 
import random

In [2]:
from random import shuffle

def my_shuffle(l):
    lc = copy.deepcopy(l)
    shuffle(lc)
    return lc

target_layer = -2
path_header = '/Users/fm/Google Drive/Colab Notebooks/'
low_threshold = 73.5
floor_threshold = 38.5


num_repeats = 4 # repeats for each condition
num_practices = 1 
num_repeats_practices = 5
num_participants = 48
projection_method = 'tsne'
elta = 0.001


candidate_materials = [
        {'folder': 'cifar10-resnet', 
         'data_name': 'cifar10', 
         'num_points': '5k', 
         'num_points_num': 5000, 
         'num_classes': 10,
         'num_folds' : 2,
         'epochs': 150
        },
        {'folder': 'mnists-cnn', 
         'data_name': 'mnist', 
         'num_points': '2k', 
         'num_points_num': 2000, 
         'num_classes': 10,
         'num_folds' : 5,
         'epochs': 50
        },
        {'folder': 'mnists-cnn', 
         'data_name': 'fmnist', 
         'num_points': '2k',
         'num_points_num': 2000, 
         'num_classes': 10,
         'num_folds' : 5,
         'epochs': 50
        },
        {'folder': 'babi-memnn', 
         'data_name': 'babi-q1', 
         'num_points': '1k', 
         'num_points_num': 1000, 
         'num_classes': 6,
         'num_folds' : 1,
         'epochs': 200
        },
        {'folder': 'babi-rnn', 
         'data_name': 'babi-q2', 
         'num_points': '1k', 
         'num_points_num': 1000, 
         'num_classes': 6,
         'num_folds' : 1,
         'epochs': 25
        }
]

experiment_factors = [
    {'factor': 'stereoscopy', 'levels': ['yes', 'no']},
    {'factor': 'motion', 'levels': ['yes', 'no']},
    {'factor': 'perspective', 'levels': ['yes', 'no']},
    {'factor': 'shading', 'levels': ['flat', 'shading-1', 'shading-2']}
]

num_sessions = 4

experiment_flow = [ 'pre-ques', 
                   'overview', 
                   'introduction', 
                   \
                   'session1', 
                   'practice1', 
                   'main1', 
                   'feedback1',
                   'break1',
                   \
                   'session2', 
                   'practice2', 
                   'main2', 
                   'feedback2',
                   'break2',
                   \
                   'session3', 
                   'practice3', 
                   'main3', 
                   'feedback3',
                   'break3',
                   \
                   'session4', 
                   'practice4', 
                   'main4', 
                   'feedback4',
                   'break4',
                   \
                   'post-ques'
                  ]

In [3]:
# build accuracy bins
material_holder_all = []
material_holder_flat = []
for d in candidate_materials:
    
    # print(d)
    
    accuracy_buckets = []
    accuracy_flat = []
    
    for e in np.arange(d['epochs']):
        
        #print(d['data_name'])
        
        for f in np.arange(d['num_folds']):
            
            file_name = d['data_name'] + '-e%04d'%(e+1) + '-f%d'%f + '-' + \
                    d['num_points'] + '-l%d'%target_layer + '-' + \
                    projection_method + '.txt'
            local_path = d['folder'] + '/saved_projections/' + file_name
            file_path = path_header + local_path
            
            #print(file_path)
            
            # if(os.path.isfile(file_path)):
            with open(file_path, 'rb') as fi:
                while True:
                    line = fi.readline()
                    if not line:
                        break
                    line_str = line[:-1].decode("utf-8") 
                    # print(line_str)
            
                    if line_str == str('test_accuracy'):
                        acc_str = fi.readline()[:-1].decode("utf-8") 
                        acc = float(acc_str)
                          
            fi.close()
            
            item = {
                    'test_accuracy' : acc,
                    'int_accuracy' : int(acc * 100),
                    'referred_file': file_name,
                    'referred_path': local_path, 
                    'data_name': d['data_name']
                }
            
            # throw out two low...
            if acc * 100 > floor_threshold:
                accuracy_buckets.append(item)
                
                
    accuracy_buckets = sorted(accuracy_buckets, key=lambda x: x['test_accuracy'])
    accuracy_flat = copy.deepcopy(accuracy_buckets)
    lowest_int_acc = accuracy_buckets[0]['int_accuracy']
    
    threshold_int_acc = 100
    for a in accuracy_buckets:
            if a['int_accuracy'] > low_threshold and a['int_accuracy'] < threshold_int_acc :
                threshold_int_acc = a['int_accuracy'] 
    upper_int_acc = accuracy_buckets[-1]['int_accuracy'] + 1
    accuracy_bins = {}
    #print(threshold_int_acc)
    
    for b in np.arange(lowest_int_acc, upper_int_acc):
        accuracy_bins[str(b)] = []
        
    for ab in accuracy_buckets:
        # print(ab)
        accuracy_bins[str(ab['int_accuracy'])].append(ab)
    print(upper_int_acc)
    #print(lower_int_acc)
    stats = [str(len(ab[1])) +':'+str(ab[0]) if len(ab[1]) > 0 else 0 for ab in accuracy_bins.items()]
    print(stats)
    material_holder_all.append({
                                'data_name': d['data_name'], 
                                'accuracy_stats': accuracy_bins,
                                'accuracy_flat': accuracy_flat,
                                'lowest_int_acc': lowest_int_acc, 
                                'threshold_int_acc': threshold_int_acc, 
                                'upper_int_acc': upper_int_acc,
                                'step': (upper_int_acc - int(low_threshold)) / float(num_repeats - 1),
                                'num_points': d['num_points_num'] ,
                                'num_classes': d['num_classes'] 
                                })
                    

92
['1:39', '1:40', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '1:59', '3:60', 0, 0, 0, 0, '2:65', '1:66', '1:67', 0, '1:69', '1:70', '2:71', '1:72', '4:73', '3:74', '2:75', '5:76', '7:77', '9:78', '11:79', '20:80', '15:81', '20:82', '16:83', '19:84', '13:85', '3:86', 0, 0, '32:89', '99:90', '7:91']
100
['7:97', '71:98', '172:99']
95
['2:83', '1:84', '3:85', '1:86', '5:87', '7:88', '16:89', '21:90', '64:91', '101:92', '27:93', '2:94']
93
['1:39', '1:40', 0, '1:42', '1:43', 0, 0, 0, '1:47', '3:48', '16:49', '19:50', '11:51', '2:52', '5:53', '2:54', '1:55', '1:56', 0, 0, 0, '1:60', 0, '1:62', 0, 0, '1:65', 0, 0, '1:68', '1:69', 0, 0, '3:72', '6:73', '3:74', '1:75', '3:76', '1:77', 0, '4:79', '2:80', '5:81', '8:82', '4:83', '9:84', '10:85', '7:86', '7:87', '12:88', '11:89', '16:90', '14:91', '2:92']
45
['2:38', '2:39', '4:40', '3:41', '3:42', '3:43', '1:44']


In [4]:
# group different accuracy
material_holder = [m for m in material_holder_all if m['data_name'] == 'cifar10' or m['data_name'] == 'babi-q1']
material_holder_folders = []

for m in material_holder:
    bins = {}
    start = low_threshold
    step = m['step']

    for ib in np.arange(num_repeats):
        bins[str(ib)] = []   
       
    i = 0
    for inst in m['accuracy_flat']:
        if inst['test_accuracy'] * 100 < low_threshold: # for each in ab
            inst['bin_lower'] = floor_threshold
            inst['bin_upper'] = low_threshold
            inst['step'] = step
            inst['bin_index'] = 0
            bins['0'].append(inst)
        else:
            index = int((inst['test_accuracy'] * 100 - low_threshold) / float(step)) + 1
            #print(index)
            inst['bin_lower'] = low_threshold + (index - 1)* step
            inst['bin_upper'] = low_threshold + index * step
            inst['step'] = step
            inst['bin_index'] = index
            #print(index)
            bins[str(index)].append(inst)
                
    material_holder_folders.append({'data_name': m['data_name'], 
                                    'num_classes': m['num_classes'], 
                                    'num_points': m['num_points'],
                                    'step': step, 'bins': bins})   

#print(material_holder_folders)

In [5]:
# 188
print(material_holder_folders[1]['step'], material_holder_folders[1]['data_name'])
[print(len(e[1]), str([i['test_accuracy'] for i in e[1]])) for e in material_holder_folders[1]['bins'].items()]

print()

# 300
print(material_holder_folders[0]['step'], material_holder_folders[0]['data_name'])
[print(len(e[1]), str([i['test_accuracy'] for i in e[1]])) for e in material_holder_folders[0]['bins'].items()]

6.666666666666667 babi-q1
74 [0.394, 0.405, 0.427, 0.439, 0.476, 0.485, 0.488, 0.488, 0.49, 0.491, 0.491, 0.492, 0.492, 0.494, 0.496, 0.496, 0.497, 0.497, 0.497, 0.498, 0.498, 0.499, 0.499, 0.499, 0.5, 0.5, 0.5, 0.5, 0.5, 0.501, 0.504, 0.505, 0.505, 0.506, 0.506, 0.507, 0.507, 0.508, 0.508, 0.508, 0.508, 0.509, 0.509, 0.51, 0.51, 0.511, 0.511, 0.512, 0.512, 0.514, 0.515, 0.515, 0.515, 0.516, 0.522, 0.526, 0.53, 0.531, 0.531, 0.531, 0.534, 0.542, 0.548, 0.559, 0.564, 0.6, 0.625, 0.658, 0.681, 0.697, 0.722, 0.724, 0.726, 0.731]
17 [0.735, 0.739, 0.739, 0.739, 0.739, 0.743, 0.747, 0.748, 0.753, 0.762, 0.764, 0.767, 0.776, 0.79, 0.79, 0.792, 0.795]
45 [0.802, 0.803, 0.81, 0.813, 0.813, 0.817, 0.819, 0.82, 0.82, 0.823, 0.824, 0.824, 0.826, 0.828, 0.828, 0.834, 0.837, 0.838, 0.839, 0.841, 0.841, 0.842, 0.842, 0.843, 0.843, 0.844, 0.847, 0.848, 0.852, 0.853, 0.854, 0.854, 0.854, 0.856, 0.857, 0.857, 0.857, 0.859, 0.86, 0.862, 0.863, 0.863, 0.864, 0.865, 0.866]
62 [0.871, 0.871, 0.874, 0.874, 

[None, None, None, None]

In [6]:
# permutation function
def permutation(lst): 
  
    # If lst is empty then there are no permutations 
    if len(lst) == 0: 
        return [] 
  
    # If there is only one element in lst then, only 
    # one permuatation is possible 
    if len(lst) == 1: 
        return [lst] 
  
    # Find the permutations for lst if there are 
    # more than 1 characters 
  
    l = [] # empty list that will store current permutation 
  
    # Iterate the input(lst) and calculate the permutation 
    for i in range(len(lst)): 
        m = lst[i] 
  
        # Extract lst[i] or m from the list.  remLst is 
        # remaining list 
        remLst = lst[:i] + lst[i+1:] 
  
        # Generating all permutations where m is first 
        # element 
        for p in permutation(remLst): 
            l.append([m] + p) 
    return l 

In [7]:
# generating VR and non-VR permutation
sm_control_all = []
for s in experiment_factors[0]['levels']:
    for m in experiment_factors[1]['levels']:
        one_flow = { experiment_factors[0]['factor']: s, experiment_factors[1]['factor']: m }
        sm_control_all.append(one_flow)

sm_control_permutation = []
for l in permutation(sm_control_all):
    sm_control_permutation.append(l)
    
sm_control = []
for p in sm_control_permutation:
    flag = 0
    for i in np.arange(len(p)):
        if i + 1 < len(p):
            if  p[i]['stereoscopy'] == p[i + 1]['stereoscopy']:
                flag += 1
                
    if flag == 0:
        sm_control.append(p)
        
# print(len(sm_control))
# print(sm_control)

In [8]:
# the rest two factors within a session

len(sm_control)

8

In [78]:
user_profiles = []
sm_control_shuffled = sm_control#my_shuffle(sm_control)
u_i = 0

for u in np.arange(num_participants):
    u_profile = []
    u_sm_control = sm_control_shuffled[u % len(sm_control_shuffled)]        
    
    s_i = 0
    
    for sm in np.arange(len(u_sm_control)):
        shading_perspective_practice = []
        perspective_bag1 = my_shuffle(experiment_factors[2]['levels'])
        for p1 in perspective_bag1:
            shading_bag1 = my_shuffle(experiment_factors[3]['levels'])
            for sd1 in shading_bag1:
                one_flow1 = { experiment_factors[2]['factor']: p1, experiment_factors[3]['factor']: sd1, 'remark': ''}
                shading_perspective_practice.append(one_flow1)
    
        shading_perspective_main = []
        perspective_bag2 = my_shuffle(experiment_factors[2]['levels'])
        for p2 in perspective_bag2:
            shading_bag2 = my_shuffle(experiment_factors[3]['levels'])
            for sd2 in shading_bag2:
                one_flow2 = { experiment_factors[2]['factor']: p2, experiment_factors[3]['factor']: sd2, 'remark': ''}
                shading_perspective_main.append(one_flow2)
            
        num_main = 0
        num_practice = 0
        session = {
                  'session_num': s_i,
                  'factors_prior': copy.deepcopy(u_sm_control[s_i]),
                  'factors_practice': copy.deepcopy(shading_perspective_practice),
                  'factors_main': copy.deepcopy(shading_perspective_main),
                  }
        s_i += 1
        u_profile.append(session)
    
    
    # in-place shuffle to shuffle the order of data
    # shuffle(material_holder_folders)
        
    if (u_i % 8) % 2 != (int(u_i / 16) % 2):
        data_name_seed = 1
    else:
        data_name_seed = 0

    if u_i == 0:
        data_name_seed = 1
        
    for s in u_profile:        
        if s['session_num'] == 0 or s['session_num'] == 1:
            data_name_index = (data_name_seed + 1) % 2
        else:
            data_name_index = (data_name_seed + 0) % 2

        repeated_samples = copy.deepcopy(material_holder_folders[data_name_index]['bins'])
        s['data_name'] = copy.deepcopy(material_holder_folders[data_name_index]['data_name'])
        
        ################################################
        #  main trials 
        ################################################
        # add two fake conditions
        max_num_factors_within_a_session = -1
        position_seed = random.random()
        position = 0
        
        if position_seed > 0.67:
            position = len(s) + 1
        elif position_seed > 0.33:
            position = int(len(s) / 2) + 1
            
        if s['factors_prior']['motion'] == 'no':
            s['factors_practice'].insert(position, {
                    'stereoscopy': s['factors_prior']['stereoscopy'],
                    'motion': 'no',
                    'shading': 'no',
                    'perspective': 'no',
                    'remark': '2d'
                })
            s['factors_main'].insert(position, {
                    'stereoscopy': s['factors_prior']['stereoscopy'],
                    'motion': 'no',
                    'shading': 'no',
                    'perspective': 'no',
                    'remark': '2d'
                })
            assert len(s['factors_main']) == len(s['factors_practice']), 'Weird issues..'
        
        max_num_factors_within_a_session = len(s['factors_main'])
          
      
        sample_offset = (s['session_num'] % 2) * max_num_factors_within_a_session
        
     
        # in-place to shuffle each bin for each participant
        for t in repeated_samples.items():
            shuffle(t[1])
        
        # generate N trials uniformly from bins for each factor
        c_i = 0
        for c in s['factors_main']:
            main_trials = []
            
            # sample for repeated trials
            bag = []
            for b in repeated_samples.items():
                bag.append(copy.deepcopy(b[1][c_i + sample_offset]))
            
            #randomize the order
            shuffle(bag)
            for b in bag:
                main_trials.append(copy.deepcopy(b))

            c['trials'] = copy.deepcopy(main_trials)
            
            # randomize the order of accuracy
            # shuffle(c['trials'])
        
            c_i += 1        
        
        # put into the main trial factors
        main_trials_combo = [] 
        for c in s['factors_main']:
            trial = {
                 'stereoscopy': s['factors_prior']['stereoscopy'],
                 'motion': s['factors_prior']['motion'],
                 'shading': c['shading'], 
                 'perspective': c['perspective'], 
                 'remark': c['remark']
            }
            for ti in c['trials']:
                trial['trial'] = copy.deepcopy(ti)
                main_trials_combo.append(copy.deepcopy(trial)) 
        
        # put trials with main factors
        s['factors_main_trials'] = main_trials_combo
        
        num_main = len(s['factors_main_trials'])
        ################################################
        #  practice trials 
        ################################################
        # sample N trials for practice trials
        practice_trials_bag1 = [] # need one for each
        practice_trials_bag2 = [] # need one for each
        
        
        # random sample to align the length
        p_count = 0
        
        for t in repeated_samples.items():
            # max_num_factors_within_a_session * 2 for main trials
            # 1 for practice trials
            if len(t[1]) > max_num_factors_within_a_session * 2:
                pt1 = copy.deepcopy(t[1][max_num_factors_within_a_session * 2])
                pt2 = copy.deepcopy(t[1][max_num_factors_within_a_session * 2 + 1])
                pt3 = copy.deepcopy(t[1][max_num_factors_within_a_session * 2 + 2])
                practice_trials_bag1.append(pt1)
                practice_trials_bag1.append(pt2)
                practice_trials_bag2.append(pt3)
                p_count+=1     
   
        # assert p_count == len(s['factors_practice']), 'Ooops, not enough practice trials'
        practice_trials_combo = [] 
        # put with the practice trials
        shuffle(practice_trials_bag1)
        shuffle(practice_trials_bag2)
        c_i = 0
        c_c = 0
        
        if u_i == 0: # id == 0 for screenshot, not used in the experiment
            for c in s['factors_practice']:
                for l in practice_trials_bag1[:3]:
                    trial = {
                     'stereoscopy': s['factors_prior']['stereoscopy'],
                     'motion': s['factors_prior']['motion'],
                     'shading': c['shading'], 
                     'perspective': c['perspective'], 
                     'remark': c['remark']
                    }
                    trial['trial'] = l
                    practice_trials_combo.append(copy.deepcopy(trial))
                    c_c += 1
                    print(l['test_accuracy'])
                print()
        else:
            for c in s['factors_practice']:
                # shuffle practice trials
                trial1 = {
                     'stereoscopy': s['factors_prior']['stereoscopy'],
                     'motion': s['factors_prior']['motion'],
                     'shading': c['shading'], 
                     'perspective': c['perspective'], 
                     'remark': c['remark']
                }

                trial1['trial'] = practice_trials_bag1[c_i]   
                practice_trials_combo.append(copy.deepcopy(trial1))

                #print(trial1['trial']['test_accuracy'])
                if trial1['remark'] == "2d" :
                    trial2 = copy.deepcopy(trial1)
                    index = random.randrange(0, len(practice_trials_bag2))
                    trial2['trial'] = practice_trials_bag2[index] 
                    practice_trials_combo.append(copy.deepcopy(trial2))
                    c_c += 1

                c_i += 1
                c_c += 1
    
        num_practice = c_c
        
        s['factors_practice_trials'] = practice_trials_combo    
        s["num_practice"] = num_practice
        s["num_main"] = num_main
        s["num_repeats"] = num_repeats
        s['num_classes'] = material_holder_folders[data_name_index]['num_classes']
        s['num_points'] = material_holder_folders[data_name_index]['num_points']
        
    profile = {"participantIndex": u_i, "participantID" : "P%d" % u_i, "projectionMethod": projection_method, 
                "sessions": u_profile}
    u_i += 1
    user_profiles.append(profile)

0.7954
0.9092
0.9094

0.7954
0.9092
0.9094

0.7954
0.9092
0.9094

0.7954
0.9092
0.9094

0.7954
0.9092
0.9094

0.7954
0.9092
0.9094

0.906
0.7162
0.7914

0.906
0.7162
0.7914

0.906
0.7162
0.7914

0.906
0.7162
0.7914

0.906
0.7162
0.7914

0.906
0.7162
0.7914

0.901
0.748
0.841

0.901
0.748
0.841

0.901
0.748
0.841

0.901
0.748
0.841

0.901
0.748
0.841

0.901
0.748
0.841

0.901
0.748
0.841

0.515
0.824
0.888

0.515
0.824
0.888

0.515
0.824
0.888

0.515
0.824
0.888

0.515
0.824
0.888

0.515
0.824
0.888

0.515
0.824
0.888



In [79]:
u_i = 0
n_n_c = 0
n_y_b = 0
n_n_b = 0
n_n_c = 0
for u in user_profiles:
    if u_i <= 48:
        for s in u["sessions"]:
            print(u['participantIndex'], s['session_num'], s["factors_prior"]['stereoscopy'] + "-" + s["factors_prior"]['motion'] + " "  + s["data_name"])
            ps = ""
            for c in s['factors_main_trials']:
                    ps += c['perspective'] +"-"+ c['shading'] +"/"
            print(ps)
            sss = s["factors_prior"]['stereoscopy'] + "-" + s["factors_prior"]['motion'] + "-" + s['data_name']
            if sss == 'no-no-cifar10':
                n_n_c += 1
            if sss == 'no-yes-babi-q1':
                n_y_b += 1
            if sss == 'no-no-babi-q1':
                n_n_b += 1
            
    u_i += 1
    print()

print(n_n_c)
print(n_y_b)
print(n_n_b)

0 0 yes-yes cifar10
no-shading-2/no-shading-2/no-shading-2/no-shading-2/no-shading-1/no-shading-1/no-shading-1/no-shading-1/no-flat/no-flat/no-flat/no-flat/yes-shading-1/yes-shading-1/yes-shading-1/yes-shading-1/yes-flat/yes-flat/yes-flat/yes-flat/yes-shading-2/yes-shading-2/yes-shading-2/yes-shading-2/
0 1 no-yes cifar10
yes-flat/yes-flat/yes-flat/yes-flat/yes-shading-1/yes-shading-1/yes-shading-1/yes-shading-1/yes-shading-2/yes-shading-2/yes-shading-2/yes-shading-2/no-flat/no-flat/no-flat/no-flat/no-shading-1/no-shading-1/no-shading-1/no-shading-1/no-shading-2/no-shading-2/no-shading-2/no-shading-2/
0 2 yes-no babi-q1
no-no/no-no/no-no/no-no/yes-shading-1/yes-shading-1/yes-shading-1/yes-shading-1/yes-shading-2/yes-shading-2/yes-shading-2/yes-shading-2/yes-flat/yes-flat/yes-flat/yes-flat/no-shading-1/no-shading-1/no-shading-1/no-shading-1/no-shading-2/no-shading-2/no-shading-2/no-shading-2/no-flat/no-flat/no-flat/no-flat/
0 3 no-no babi-q1
no-shading-1/no-shading-1/no-shading-1/no-sha

In [70]:
print(user_profiles[0:2])

[{'participantIndex': 0, 'participantID': 'P0', 'projectionMethod': 'tsne', 'sessions': [{'session_num': 0, 'factors_prior': {'stereoscopy': 'no', 'motion': 'no'}, 'factors_practice': [{'perspective': 'yes', 'shading': 'flat', 'remark': ''}, {'perspective': 'yes', 'shading': 'shading-2', 'remark': ''}, {'perspective': 'yes', 'shading': 'shading-1', 'remark': ''}, {'stereoscopy': 'no', 'motion': 'no', 'shading': 'no', 'perspective': 'no', 'remark': '2d'}, {'perspective': 'no', 'shading': 'shading-2', 'remark': ''}, {'perspective': 'no', 'shading': 'shading-1', 'remark': ''}, {'perspective': 'no', 'shading': 'flat', 'remark': ''}], 'factors_main': [{'perspective': 'yes', 'shading': 'shading-2', 'remark': '', 'trials': [{'test_accuracy': 0.8968, 'int_accuracy': 89, 'referred_file': 'cifar10-e0118-f0-5k-l-2-tsne.txt', 'referred_path': 'cifar10-resnet/saved_projections/cifar10-e0118-f0-5k-l-2-tsne.txt', 'data_name': 'cifar10', 'bin_lower': 86.16666666666667, 'bin_upper': 92.5, 'step': 6.333

In [73]:
import os
cwd = os.getcwd()
path_headers = "/".join(cwd.split("/")[:-2]) # the github repo

In [74]:
import json


u_i = 0
for u in user_profiles:
    path = path_headers + "/experiment/datasets/participants/" + "P%d"%u_i + ".json"
    with open(path, 'w') as output_file:
        json.dump(u, output_file)
    u_i += 1